In [1]:
# !pip install altair vega_datasets
# !pip install jupyter-dash

In [189]:
import pandas as pd 
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import re 
import string

import altair as alt 
from vega_datasets import data
from jupyter_dash import JupyterDash

# Accounts mapping

In [190]:
file = 'Account_Mapping_NoCat.csv' 
accounts_df = pd.read_csv(file)
accounts_df = accounts_df.drop_duplicates() 
accounts_df = accounts_df.rename(columns={'id': 'mapped_acoount_id'})
accounts_df['facebook_id'] = accounts_df['facebook_id'].fillna(0)
accounts_df['facebook_id'] = accounts_df['facebook_id'].apply(np.int64)

accounts_df['twitter_id'] = accounts_df['twitter_id'].fillna(0)
accounts_df['twitter_id'] = accounts_df['twitter_id'].apply(np.int64)

accounts_df['instagram_id'] = accounts_df['instagram_id'].fillna(0)
accounts_df['instagram_id'] = accounts_df['instagram_id'].apply(np.int64)


In [191]:
#Clean text  

#Need to skip removing punctuation -> e.g. BKKMENU.COM
keep_punct = ['-', '&', '_','.']  

def remove_punctuations(text):
    new_text = ''
    for char in text:  
        if char in keep_punct:
            char
        else:    
            for punctuation in string.punctuation: 
                    char = char.translate(str.maketrans('', '', string.punctuation))    
        new_text += char 
    return new_text  

accounts_df['account_name'] = accounts_df['account_name'].str.strip()  
accounts_df['account_name'] = accounts_df['account_name'].replace('\n', '')  
accounts_df['account_name'] = accounts_df['account_name'].apply(lambda x: re.sub(r' +', ' ', x)) 
accounts_df['account_name'] = accounts_df['account_name'].apply(remove_punctuations)
accounts_df['account_name'] = [word.upper() for word in accounts_df['account_name']] 

check = pd.DataFrame(data=accounts_df['account_name'].unique().flatten()) 
check.to_csv('List_of_cleaned_account_names.csv', encoding='utf8') 

## Create DF for each id & name of each platform  

Since we found that 1 mapped accont id can be with more >= 1 different names (but they are the same person)
We will use the most minimal name of each account for each platform

In [192]:
#Select the most minimal name for each fb id
fb_list = accounts_df.groupby('facebook_id')['account_name'].min().to_frame().reset_index() 
yt_list = accounts_df.groupby('youtube_id')['account_name'].min().to_frame().reset_index() 
tw_list = accounts_df.groupby('twitter_id')['account_name'].min().to_frame().reset_index() 
ig_list = accounts_df.groupby('instagram_id')['account_name'].min().to_frame().reset_index()

# Facebook

In [193]:
appended_data = []
for Q in range(4):
    file = 'Facebook_Data_Q' + str(Q+1) + '.csv' 
    df = pd.read_csv(file, error_bad_lines = False)
    appended_data.append(df) 

fb_df = pd.concat(appended_data)
fb_df = fb_df.drop_duplicates()

/Users/scharoensawa/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/scharoensawa/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [194]:
fb_df['created_at'] = pd.to_datetime(fb_df['created_at'])
fb_df['updated_at'].fillna('') 
fb_df['updated_at'] = pd.to_datetime(fb_df['updated_at'], errors='coerce')  #https://newbedev.com/how-do-i-convert-strings-in-a-pandas-data-frame-to-a-date-data-type
fb_df['day_of_week'] = fb_df['created_at'].dt.day_name()
fb_df['hour'] = fb_df['created_at'].dt.hour
fb_df['week'] = fb_df['created_at'].dt.week
fb_df['month'] = fb_df['created_at'].astype('datetime64[s]').dt.month 

<ipython-input-194-a63aa6dd5067>:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  fb_df['week'] = fb_df['created_at'].dt.week


In [195]:
#Clean text  
fb_df['account_display_name'] = fb_df['account_display_name'].fillna('')
fb_df['account_display_name'] = fb_df['account_display_name'].str.strip() 
fb_df['account_display_name'] = fb_df['account_display_name'].replace('\n', '')  
fb_df['account_display_name'] = fb_df['account_display_name'].apply(lambda x: re.sub(r' +', ' ', x)) 
fb_df['account_display_name'] = [word.upper() for word in fb_df['account_display_name']] 


fb_df['account_username'] = fb_df['account_username'].fillna('')
fb_df['account_username'] = fb_df['account_username'].str.strip() 
fb_df['account_username'] = fb_df['account_username'].replace('\n', '')  
fb_df['account_username'] = fb_df['account_username'].apply(lambda x: re.sub(r' +', ' ', x))
fb_df['account_username'] = [word.upper() for word in fb_df['account_username']] 

In [196]:
fb_df.groupby(['account_id', 'day_of_week', 'hour', 'month', 'week'])['engagement'].sum().to_frame().reset_index()
fb_df

,post_id,channel,account_id,account_username,account_display_name,link,created_at,message,like,love,...,neutral,negative,fan,updated_at,is_photo,category,day_of_week,hour,week,month
0,100106458256720_100110608256305,facebook,100106458256720,,ครูพี่หมี ติวภาษาอังกฤษ,https://www.facebook.com/album.php?fbid=100110...,2020-02-17 12:26:55,NaN,0,0,...,0,0,68350,NaT,0,\N,Monday,12,8,2
1,100106458256720_100110984922934,facebook,100106458256720,,ครูพี่หมี ติวภาษาอังกฤษ,https://www.facebook.com/album.php?fbid=100110...,2020-02-17 12:28:19,NaN,20,7,...,0,0,68350,2020-12-15 18:16:25,0,\N,Monday,12,8,2
2,100106458256720_100111091589590,facebook,100106458256720,,ครูพี่หมี ติวภาษาอังกฤษ,https://www.facebook.com/100106458256720/posts...,2020-02-17 12:28:19,NaN,20,7,...,0,0,68350,2020-12-15 18:16:26,0,\N,Monday,12,8,2
3,100106458256720_100130248254341,facebook,100106458256720,,ครูพี่หมี ติวภาษาอังกฤษ,https://www.facebook.com/album.php?fbid=100130...,2020-02-17 13:14:31,NaN,136,4,...,0,0,68350,2020-12-15 18:16:49,0,\N,Monday,13,8,2
4,100106458256720_100857674848265,facebook,100106458256720,,ครูพี่หมี ติวภาษาอังกฤษ,https://www.facebook.com/100106458256720/posts...,2020-02-18 15:27:18,NaN,5,0,...,0,0,68350,2020-12-15 18:28:00,0,\N,Tuesday,15,8,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937847,997722787054306_1725574547602456,facebook,997722787054306,SOPITCHITCHAT โสภิตชิทแชท,SOPITCHITCHAT โสภิตชิทแชท,https://www.facebook.com/sopitchitchat/photos/...,2020-12-29 16:28:09,ยอมรับ....ความจริง \nยอมรับ....สิ่งที่เกิดขึ้น...,553,45,...,0,0,115849,2021-01-06 18:49:57,0,\N,Tuesday,16,53,12
937848,997722787054306_1726203047539606,facebook,997722787054306,SOPITCHITCHAT โสภิตชิทแชท,SOPITCHITCHAT โสภิตชิทแชท,https://www.facebook.com/sopitchitchat/photos/...,2020-12-30 11:57:58,<< อ่อนไหวไม่ใช่อ่อนแอ >>\n\nเรื่องบางเรื่อง....,602,45,...,0,0,115849,2021-01-06 18:49:58,0,travel,Wednesday,11,53,12
937849,997722787054306_343785250044127,facebook,997722787054306,,SOPITCHITCHAT โสภิตชิทแชท,https://www.facebook.com/sopitchitchat/videos/...,2020-10-17 13:06:33,บทสวดน้อมนำบุญ ส่งต่อ\n,11,1,...,0,0,115455,NaT,1,\N,Saturday,13,42,10
937850,997722787054306_355057459246954,facebook,997722787054306,,SOPITCHITCHAT โสภิตชิทแชท,https://www.facebook.com/sopitchitchat/videos/...,2020-10-17 13:06:33,กราบนมัสการ\n,11,0,...,0,0,115455,NaT,1,\N,Saturday,13,42,10


## DF for graph 1 & 2

In [197]:
#Each account, month -> select max created at
a = fb_df.groupby(['account_id', 'month'])['created_at'].max().to_frame()

#Each account, month, max_created at -> select max updated at
t = fb_df.merge(a, on = ['account_id', 'created_at'], how = 'inner' )
tt = t.groupby(['account_id', 'month'])['updated_at'].max().to_frame()
latest_update_of_max_create = a.merge(tt, on = ['account_id', 'month'], how = 'inner' ).reset_index() 

latest_fan = fb_df.merge(latest_update_of_max_create, on = ['account_id', 'month', 'created_at','updated_at'], how = 'inner' )
b = latest_fan[['account_id', 'fan', 'month']].drop_duplicates() 

#Filter condition
#Fake Account: accounts with (high) followers, but no view or engagement (Assumption: accounts that buy followers) 
cleaned_fb_df = fb_df.groupby(['account_id', 'month'])['engagement'].sum()
cleaned_fb_df = cleaned_fb_df.to_frame().reset_index()
cleaned_fb_df = b.merge(cleaned_fb_df, on = ['account_id', 'month'])

cleaned_fb_df['fan'] = pd.to_numeric(cleaned_fb_df['fan'],errors='coerce') 
filter_criteria = ((cleaned_fb_df['fan'] > 0) & (cleaned_fb_df['engagement'] > 0)) 
cleaned_fb_df = cleaned_fb_df.loc[filter_criteria]

cleaned_fb_df['reactions'] = cleaned_fb_df['engagement']
cleaned_fb_df = cleaned_fb_df.drop(columns = 'engagement')
cleaned_fb_df['reactions_per_fan'] = cleaned_fb_df['reactions']/cleaned_fb_df['fan']
cleaned_fb_df = cleaned_fb_df.drop_duplicates()

fb_df_graph1_2 = cleaned_fb_df
fb_df_graph1_2 = fb_df_graph1_2[fb_df_graph1_2['fan'].notna()].copy()
fb_df_graph1_2

,account_id,fan,month,reactions,reactions_per_fan
0,100106458256720,68350.0,2,368,0.005384
1,100106458256720,68350.0,3,1008,0.014748
2,1001088136643378,6827.0,1,1401,0.205215
3,1001088136643378,6827.0,2,1119,0.163908
4,1001088136643378,7044.0,3,532,0.075525
...,...,...,...,...,...
22539,995302490483328,86918.0,11,14278,0.164270
22540,995302490483328,88267.0,12,23614,0.267529
22541,997722787054306,115702.0,10,65056,0.562272
22542,997722787054306,115847.0,11,72939,0.629615


## DF for breakdown graph 1 & 2

In [198]:
fb_df['created_date'] = fb_df['created_at'].dt.date.astype(str)    

#Each account, month, date -> select max created at
a = fb_df.groupby(['account_id', 'month', 'created_date'])['created_at'].max().to_frame()

#Each account, date, max_created at -> select max updated at
t = fb_df.merge(a, on = ['account_id', 'created_date', 'created_at'], how = 'inner' )
tt = t.groupby(['account_id', 'created_date'])['updated_at'].max().to_frame()
latest_update_of_max_create = a.merge(tt, on = ['account_id', 'created_date'], how = 'inner' ).reset_index() 

latest_fan = fb_df.merge(latest_update_of_max_create, on = ['account_id', 'created_date', 'created_at','updated_at'], how = 'inner' )
b = latest_fan[['account_id', 'fan', 'month', 'created_date']].drop_duplicates() 

#Filter condition
#Fake Account: accounts with (high) followers, but no view or engagement (Assumption: accounts that buy followers) 
cleaned_fb_df = fb_df.groupby(['account_id', 'month', 'created_date'])['engagement'].sum()
cleaned_fb_df = cleaned_fb_df.to_frame().reset_index()
cleaned_fb_df = b.merge(cleaned_fb_df, on = ['account_id', 'month', 'created_date'])
 
    
cleaned_fb_df['fan'] = pd.to_numeric(cleaned_fb_df['fan'],errors='coerce') 
filter_criteria = ((cleaned_fb_df['fan'] > 0) & (cleaned_fb_df['engagement'] > 0)) 
cleaned_fb_df = cleaned_fb_df.loc[filter_criteria] 


cleaned_fb_df['reactions'] = cleaned_fb_df['engagement']
cleaned_fb_df = cleaned_fb_df.drop(columns = 'engagement')
cleaned_fb_df['reactions_per_fan'] = cleaned_fb_df['reactions']/cleaned_fb_df['fan']
cleaned_fb_df = cleaned_fb_df.drop_duplicates()

fb_df_breakdown_graph1_2 = cleaned_fb_df
fb_df_breakdown_graph1_2 = fb_df_breakdown_graph1_2[fb_df_breakdown_graph1_2['fan'].notna()].copy()      
fb_df_breakdown_graph1_2

,account_id,fan,month,created_date,reactions,reactions_per_fan
0,100106458256720,68350.0,2,2020-02-17,363,0.005311
1,100106458256720,68350.0,2,2020-02-18,5,0.000073
3,100106458256720,68350.0,3,2020-03-09,1008,0.014748
4,1001088136643378,6827.0,3,2020-03-08,71,0.010400
5,1001088136643378,6704.0,1,2020-01-01,127,0.018944
...,...,...,...,...,...,...
393145,997722787054306,115858.0,12,2020-12-26,753,0.006499
393146,997722787054306,115860.0,12,2020-12-27,579,0.004997
393147,997722787054306,115856.0,12,2020-12-28,295,0.002546
393148,997722787054306,115849.0,12,2020-12-29,607,0.005240


## DF for graph 3

In [199]:
#DF for graph 3 
fb_df_graph3 = fb_df.merge(cleaned_fb_df, on = ['account_id', 'month'], how = 'inner' ) 
fb_df_graph3 = fb_df_graph3.groupby(['account_id', 'day_of_week', 'hour', 'month', 'week'])['engagement'].sum().to_frame().reset_index()
fb_df_graph3['reactions'] = fb_df_graph3['engagement']
fb_df_graph3 = fb_df_graph3.drop(columns = 'engagement')  
fb_df_graph3_by_account = fb_df_graph3


fb_df_graph3['reactions_by_hour'] = fb_df_graph3.groupby(['month', 'day_of_week', 'hour'])['reactions'].transform('sum') 
fb_df_graph3['total_reactions'] = fb_df_graph3.groupby(['month'])['reactions'].transform('sum') # numer_by_hour = fb_df_graph3.groupby(['day_of_week', 'hour','month', 'week'])['reactions'].sum().to_frame().reset_index() 
fb_df_graph3['pct_reactions'] = 100*fb_df_graph3['reactions_by_hour'] /fb_df_graph3['total_reactions']
fb_df_graph3 = fb_df_graph3.drop(columns = ['account_id','reactions','reactions_by_hour','total_reactions','week'])
fb_df_graph3 = fb_df_graph3.drop_duplicates()
fb_df_graph3 =  fb_df_graph3.dropna(subset=['month', 'day_of_week', 'hour'])
fb_df_graph3

,day_of_week,hour,month,pct_reactions
0,Friday,7,7,0.704825
2,Friday,8,3,0.612994
3,Friday,8,12,0.708678
4,Friday,9,1,0.707808
6,Friday,9,4,0.527705
...,...,...,...,...
73578,Wednesday,5,5,0.047379
96897,Friday,2,4,0.018382
107216,Monday,2,5,0.005453
107428,Tuesday,2,7,0.009032


# Youtube

In [200]:
YT_df = pd.read_csv('YouTube_Data.csv')
YT_df = YT_df.drop_duplicates()

YT_df['reactions'] = YT_df['like']+YT_df['dislike']+YT_df['favorite']+YT_df['comment']+YT_df['view']+YT_df['share_on_facebook']
YT_df['created_at'] = YT_df['created_at'].astype('datetime64[s]')
YT_df['day_of_week'] = YT_df['created_at'].dt.day_name()
YT_df['hour'] = YT_df['created_at'].dt.hour
YT_df['week'] = YT_df['created_at'].dt.week
YT_df['month'] = YT_df['created_at'].dt.month

<ipython-input-200-1177e24eb717>:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  YT_df['week'] = YT_df['created_at'].dt.week


In [201]:
#clean text 
YT_df['account_display_name'] = YT_df['account_display_name'].fillna('')
YT_df['account_display_name'] = YT_df['account_display_name'].str.strip() 
YT_df['account_display_name'] = YT_df['account_display_name'].replace('\n', '')  
YT_df['account_display_name'] = YT_df['account_display_name'].apply(lambda x: re.sub(r' +', ' ', x)) 
YT_df['account_display_name'] = [word.upper() for word in YT_df['account_display_name']] 

YT_df['account_username'] = YT_df['account_username'].fillna('')
YT_df['account_username'] = YT_df['account_username'].str.strip() 
YT_df['account_username'] = YT_df['account_username'].replace('\n', '')  
YT_df['account_username'] = YT_df['account_username'].apply(lambda x: re.sub(r' +', ' ', x)) 
YT_df['account_username'] = [word.upper() for word in YT_df['account_username']] 

## DF for graph 1 & 2

In [202]:
#find latest fan for each month & account
a = YT_df.groupby(['account_id', 'month'])['created_at'].max() 
latest_fan = YT_df.merge( a, on = ['account_id', 'created_at'], how = 'inner' ) 
b = latest_fan[['account_id', 'fan', 'month']].drop_duplicates()  

#Filter condition
#Fake Account: accounts with (high) followers, but no view or engagement (Assumption: accounts that buy followers) 
cleaned_yt_df = YT_df.groupby(['account_id', 'month'])['reactions'].sum()
cleaned_yt_df = cleaned_yt_df.to_frame().reset_index()
cleaned_yt_df = b.merge(cleaned_yt_df, on = ['account_id', 'month'])

#filter_criteria change \ to & becuz \ still have fan = 0  so if fan = 0 หาร react result = inf
filter_criteria = ((cleaned_yt_df['fan'] > 0) & (cleaned_yt_df['reactions'] > 0)) 
cleaned_yt_df = cleaned_yt_df.loc[filter_criteria]
cleaned_yt_df['reactions_per_fan'] = cleaned_yt_df['reactions']/cleaned_yt_df['fan']
cleaned_yt_df = cleaned_yt_df.drop_duplicates()

yt_df_graph1_2 = cleaned_yt_df   

## DF for breakdown graph 1 & 2

In [203]:
YT_df['created_date'] = YT_df['created_at'].dt.date.astype(str)    

#find latest fan for each month & account
a = YT_df.groupby(['account_id', 'month', 'created_date'])['created_at'].max() 
latest_fan = YT_df.merge( a, on = ['account_id', 'created_date', 'created_at'], how = 'inner' ) 
b = latest_fan[['account_id', 'fan', 'month', 'created_date']].drop_duplicates()  

#Filter condition
#Fake Account: accounts with (high) followers, but no view or engagement (Assumption: accounts that buy followers) 
cleaned_yt_df = YT_df.groupby(['account_id', 'month', 'created_date'])['reactions'].sum()
cleaned_yt_df = cleaned_yt_df.to_frame().reset_index()
cleaned_yt_df = b.merge(cleaned_yt_df, on = ['account_id', 'month', 'created_date'])

#filter_criteria change \ to & becuz \ still have fan = 0  so if fan = 0 หาร react result = inf
filter_criteria = ((cleaned_yt_df['fan'] > 0) & (cleaned_yt_df['reactions'] > 0)) 
cleaned_yt_df = cleaned_yt_df.loc[filter_criteria]
cleaned_yt_df['reactions_per_fan'] = cleaned_yt_df['reactions']/cleaned_yt_df['fan']
cleaned_yt_df = cleaned_yt_df.drop_duplicates()

yt_df_breakdown_graph1_2 = cleaned_yt_df   

## DF for graph 3

In [204]:
cleaned_yt_df_forgraph3 = cleaned_yt_df.drop(columns = ['fan','reactions','reactions_per_fan'])
cleaned_yt_df_forgraph3 = cleaned_yt_df_forgraph3.drop_duplicates()
yt_df_graph3 = YT_df.merge(cleaned_yt_df_forgraph3, on = ['account_id', 'month'], how = 'inner' ) 

yt_df_graph3 = yt_df_graph3.groupby(['account_id', 'day_of_week', 'hour', 'month', 'week'])['reactions'].sum().to_frame().reset_index()
yt_df_graph3_by_account = yt_df_graph3

     
yt_df_graph3['reactions_by_hour'] = yt_df_graph3.groupby(['month', 'day_of_week', 'hour'])['reactions'].transform('sum') 
yt_df_graph3['total_reactions'] = yt_df_graph3.groupby(['month'])['reactions'].transform('sum')
yt_df_graph3['pct_reactions'] = 100*yt_df_graph3['reactions_by_hour'] /yt_df_graph3['total_reactions']
yt_df_graph3 = yt_df_graph3.drop(columns = ['account_id','reactions','reactions_by_hour','total_reactions', 'week'])
yt_df_graph3 = yt_df_graph3.drop_duplicates()
yt_df_graph3 = yt_df_graph3.dropna(subset=['month', 'day_of_week', 'hour'])  

# Twitter

In [205]:
file = 'Twitter_Data.csv'
tw_df = pd.read_csv(file, encoding="utf-8")  
tw_df = tw_df.drop_duplicates()

tw_df['created_at'] = tw_df['created_at'].astype('datetime64[s]')
tw_df['day_of_week'] = tw_df['created_at'].dt.day_name()
tw_df['hour'] = tw_df['created_at'].dt.hour
tw_df['week'] = tw_df['created_at'].dt.week
tw_df['month'] = tw_df['created_at'].dt.month

<ipython-input-205-76b6f885c9b1>:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  tw_df['week'] = tw_df['created_at'].dt.week


In [206]:
#Remove emoji
#https://poopcode.com/how-to-remove-emoji-from-text-in-python/
# full version of emoji cleaner
def deEmojify(text):
    "function to remove emojis from text"
    regrex_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return regrex_pattern.sub(r'',text)

tw_df['account_display_name'] = tw_df['account_display_name'].apply(deEmojify) 
tw_df['account_username'] = tw_df['account_username'].apply(deEmojify) 

In [207]:
#clean text  
tw_df['account_display_name'] = tw_df['account_display_name'].str.strip() 
tw_df['account_display_name'] = tw_df['account_display_name'].replace('\n', '')  
tw_df['account_display_name'] = tw_df['account_display_name'].apply(lambda x: re.sub(r' +', ' ', x)) 
tw_df['account_display_name'] = [word.upper() for word in tw_df['account_display_name']] 

tw_df['account_username'] = tw_df['account_username'].str.strip() 
tw_df['account_username'] = tw_df['account_username'].replace('\n', '')  
tw_df['account_username'] = tw_df['account_username'].apply(lambda x: re.sub(r' +', ' ', x)) 
tw_df['account_username'] = [word.upper() for word in tw_df['account_username']] 

## DF for graph 1& 2

In [208]:
#find end of period by account
a = tw_df.groupby(['account_id', 'month'])['created_at'].max()
#merge to rawdata and select column fan
latest_fan = tw_df.merge( a, on = ['account_id', 'created_at'], how = 'inner' ) 
b = latest_fan[['account_id', 'fan', 'month']].drop_duplicates() 

#create reactions column
tw_df['reactions'] = tw_df['engagement'] + tw_df['reply']

#Filter condition
#Fake Account: accounts with (high) followers, but no view or engagement (Assumption: accounts that buy followers) 
cleaned_tw_df = tw_df.groupby(['account_id', 'month'])['reactions'].sum()
cleaned_tw_df = cleaned_tw_df.to_frame().reset_index()
cleaned_tw_df = b.merge(cleaned_tw_df, on = ['account_id', 'month'])
cleaned_tw_df['fan'] = pd.to_numeric(cleaned_tw_df['fan'],errors='coerce') 

filter_criteria = ((cleaned_tw_df['fan'] > 0) & (cleaned_tw_df['reactions'] > 0)) 
cleaned_tw_df = cleaned_tw_df.loc[filter_criteria]
cleaned_tw_df['reactions_per_fan'] = cleaned_tw_df['reactions']/cleaned_tw_df['fan']
cleaned_tw_df = cleaned_tw_df.drop_duplicates()

tw_df_graph1_2 = cleaned_tw_df 

## DF for breakdown graph 1 & 2

In [209]:
tw_df['created_date'] = tw_df['created_at'].dt.date.astype(str)   

#find end of period by account
a = tw_df.groupby(['account_id', 'month', 'created_date'])['created_at'].max()
#merge to rawdata and select column fan
latest_fan = tw_df.merge( a, on = ['account_id', 'created_at', 'created_date'], how = 'inner' ) 
b = latest_fan[['account_id', 'fan', 'month', 'created_date']].drop_duplicates() 

#create reactions column
tw_df['reactions'] = tw_df['engagement'] + tw_df['reply']

#Filter condition
#Fake Account: accounts with (high) followers, but no view or engagement (Assumption: accounts that buy followers) 
cleaned_tw_df = tw_df.groupby(['account_id', 'month', 'created_date'])['reactions'].sum()
cleaned_tw_df = cleaned_tw_df.to_frame().reset_index()
cleaned_tw_df = b.merge(cleaned_tw_df, on = ['account_id', 'month', 'created_date'])
cleaned_tw_df['fan'] = pd.to_numeric(cleaned_tw_df['fan'],errors='coerce') 

filter_criteria = ((cleaned_tw_df['fan'] > 0) & (cleaned_tw_df['reactions'] > 0)) 
cleaned_tw_df = cleaned_tw_df.loc[filter_criteria]
cleaned_tw_df['reactions_per_fan'] = cleaned_tw_df['reactions']/cleaned_tw_df['fan']
cleaned_tw_df = cleaned_tw_df.drop_duplicates()

tw_df_breakdown_graph1_2 = cleaned_tw_df 

## DF for graph 3 

In [210]:
#DF for graph 3 
cleaned_tw_df_forgraph3 = cleaned_tw_df.drop(columns = ['fan','reactions','reactions_per_fan'])
cleaned_tw_df_forgraph3 = cleaned_tw_df_forgraph3.drop_duplicates()
tw_df_graph3 = tw_df.merge(cleaned_tw_df_forgraph3, on = ['account_id', 'month'], how = 'inner' ) 

tw_df_graph3 = tw_df_graph3.groupby(['account_id', 'day_of_week', 'hour', 'month', 'week'])['reactions'].sum().to_frame().reset_index()
tw_df_graph3_by_account = tw_df_graph3

     
tw_df_graph3['reactions_by_hour'] = tw_df_graph3.groupby(['month', 'day_of_week', 'hour'])['reactions'].transform('sum') 
tw_df_graph3['total_reactions'] = tw_df_graph3.groupby(['month'])['reactions'].transform('sum') # numer_by_hour = fb_df_graph3.groupby(['day_of_week', 'hour','month', 'week'])['reactions'].sum().to_frame().reset_index() 
tw_df_graph3['pct_reactions'] = 100*tw_df_graph3['reactions_by_hour'] /tw_df_graph3['total_reactions']
tw_df_graph3 = tw_df_graph3.drop(columns = ['account_id','reactions','reactions_by_hour','total_reactions', 'week'])
tw_df_graph3 = tw_df_graph3.drop_duplicates() 
tw_df_graph3 = tw_df_graph3.dropna(subset=['month', 'day_of_week', 'hour'])  

# Instagram

In [211]:
ig_df = pd.read_csv('Instagram_Data.csv')
ig_df  = ig_df.drop_duplicates()

ig_df['created_at'] = pd.to_datetime(ig_df['created_at'])
ig_df['day_of_week'] = ig_df['created_at'].dt.day_name()
ig_df['hour'] = ig_df['created_at'].dt.hour
ig_df['week'] = ig_df['created_at'].dt.week
ig_df['month'] = ig_df['created_at'].dt.month


ig_df2 = ig_df.groupby(['account_id', 'day_of_week', 'hour', 'month', 'week'])['engagement'].sum().to_frame().reset_index()

df = pd.read_csv('Instagram_Data.csv')
df = df.drop_duplicates()

df['created_at'] = pd.to_datetime(df['created_at'])
df['day_of_week'] = df['created_at'].dt.day_name()
df['hour'] = df['created_at'].dt.hour
df['week'] = df['created_at'].dt.week
df['month'] = df['created_at'].dt.month


df2 = df.groupby(['account_id', 'day_of_week', 'hour', 'month', 'week'])['engagement'].sum().to_frame().reset_index()

<ipython-input-211-44a969d76252>:7: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  ig_df['week'] = ig_df['created_at'].dt.week
<ipython-input-211-44a969d76252>:19: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['week'] = df['created_at'].dt.week


In [212]:
#clean text  
ig_df['account_display_name'] = ig_df['account_display_name'].fillna('')
ig_df['account_display_name'] = ig_df['account_display_name'].str.strip() 
ig_df['account_display_name'] = ig_df['account_display_name'].replace('\n', '')  
ig_df['account_display_name'] = ig_df['account_display_name'].apply(lambda x: re.sub(r' +', ' ', x)) 
ig_df['account_display_name'] = [word.upper() for word in ig_df['account_display_name']] 

ig_df['account_username'] = ig_df['account_username'].fillna('')
ig_df['account_username'] = ig_df['account_username'].str.strip() 
ig_df['account_username'] = ig_df['account_username'].replace('\n', '')  
ig_df['account_username'] = ig_df['account_username'].apply(lambda x: re.sub(r' +', ' ', x)) 
ig_df['account_username'] = [word.upper() for word in ig_df['account_username']] 

In [213]:
#Remove emoji
#https://poopcode.com/how-to-remove-emoji-from-text-in-python/
# full version of emoji cleaner
def deEmojify(text):
    "function to remove emojis from text"
    regrex_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return regrex_pattern.sub(r'',text)

ig_df['account_display_name'] = ig_df['account_display_name'].apply(deEmojify) 
ig_df['account_username'] = ig_df['account_username'].apply(deEmojify) 

## DF for graph 1 & 2

In [214]:
#find end of period by account
a = ig_df.groupby(['account_id', 'month'])['created_at'].max()
#merge to rawdata and select column fan
latest_fan = ig_df.merge( a, on = ['account_id', 'created_at'], how = 'inner' ) 
b = latest_fan[['account_id', 'fan', 'month']].drop_duplicates() 

#create reactions column
ig_df['reactions'] = ig_df['engagement'] + ig_df['share_on_facebook']

#Filter condition
#Fake Account: accounts with (high) followers, but no view or engagement (Assumption: accounts that buy followers) 
cleaned_ig_df = ig_df.groupby(['account_id', 'month'])['reactions'].sum()
cleaned_ig_df = cleaned_ig_df.to_frame().reset_index()
cleaned_ig_df = b.merge(cleaned_ig_df, on = ['account_id', 'month'])
cleaned_ig_df['fan'] = pd.to_numeric(cleaned_ig_df['fan'],errors='coerce') 

filter_criteria = ((cleaned_ig_df['fan'] > 0) & (cleaned_ig_df['reactions'] > 0)) 
cleaned_ig_df = cleaned_ig_df.loc[filter_criteria]
cleaned_ig_df['reactions_per_fan'] = cleaned_ig_df['reactions']/cleaned_ig_df['fan']
cleaned_ig_df = cleaned_ig_df.drop_duplicates()

ig_df_graph1_2 = cleaned_ig_df 

## DF for breakdown graph 1 & 2

In [215]:
ig_df['created_date'] = ig_df['created_at'].dt.date.astype(str)   

#find end of period by account
a = ig_df.groupby(['account_id', 'month', 'created_date'])['created_at'].max()
#merge to rawdata and select column fan
latest_fan = ig_df.merge( a, on = ['account_id', 'created_at', 'created_date'], how = 'inner' ) 
b = latest_fan[['account_id', 'fan', 'month', 'created_date']].drop_duplicates() 

#create reactions column
ig_df['reactions'] = ig_df['engagement'] + ig_df['share_on_facebook']

#Filter condition
#Fake Account: accounts with (high) followers, but no view or engagement (Assumption: accounts that buy followers) 
cleaned_ig_df = ig_df.groupby(['account_id', 'month', 'created_date'])['reactions'].sum()
cleaned_ig_df = cleaned_ig_df.to_frame().reset_index()
cleaned_ig_df = b.merge(cleaned_ig_df, on = ['account_id', 'month', 'created_date'])
cleaned_ig_df['fan'] = pd.to_numeric(cleaned_ig_df['fan'],errors='coerce') 

filter_criteria = ((cleaned_ig_df['fan'] > 0) & (cleaned_ig_df['reactions'] > 0)) 
cleaned_ig_df = cleaned_ig_df.loc[filter_criteria]
cleaned_ig_df['reactions_per_fan'] = cleaned_ig_df['reactions']/cleaned_ig_df['fan']
cleaned_ig_df = cleaned_ig_df.drop_duplicates()

ig_df_breakdown_graph1_2 = cleaned_ig_df 
ig_df_breakdown_graph1_2

,account_id,fan,month,created_date,reactions,reactions_per_fan
0,1642511903,3186,1,2020-01-03,51,0.016008
1,1642511903,3186,1,2020-01-05,97,0.030446
2,1642511903,3186,1,2020-01-01,58,0.018205
3,1642511903,3186,1,2020-01-07,29,0.009102
4,1642511903,3186,1,2020-01-18,77,0.024168
...,...,...,...,...,...,...
147174,179978190,215134,12,2020-12-01,20249,0.094123
147175,179978190,215134,12,2020-12-12,24805,0.115300
147176,179978190,215134,12,2020-12-03,24111,0.112074
147177,179978190,215134,12,2020-12-14,30537,0.141944


## DF for graph 3

In [216]:
ig_df_graph3 = ig_df.merge(cleaned_ig_df, on = ['account_id', 'month'], how = 'inner' ) 
ig_df_graph3 = ig_df_graph3.groupby(['account_id', 'day_of_week', 'hour', 'month', 'week'])['engagement'].sum().to_frame().reset_index()
ig_df_graph3['reactions'] = ig_df_graph3['engagement']
ig_df_graph3 = ig_df_graph3.drop(columns = 'engagement')  
ig_df_graph3_by_account = ig_df_graph3



ig_df_graph3['reactions_by_hour'] = ig_df_graph3.groupby(['month', 'day_of_week', 'hour'])['reactions'].transform('sum') 
ig_df_graph3['total_reactions'] = ig_df_graph3.groupby(['month'])['reactions'].transform('sum') # numer_by_hour = fb_df_graph3.groupby(['day_of_week', 'hour','month', 'week'])['reactions'].sum().to_frame().reset_index() 
ig_df_graph3['pct_reactions'] = 100*ig_df_graph3['reactions_by_hour'] /ig_df_graph3['total_reactions']
ig_df_graph3 = ig_df_graph3.drop(columns = ['account_id','reactions','reactions_by_hour','total_reactions', 'week'])
ig_df_graph3 = ig_df_graph3.drop_duplicates() 
ig_df_graph3 = ig_df_graph3.dropna(subset=['month', 'day_of_week', 'hour'])  

# Merge DFs

## Main

In [369]:
#DF names of each platform
fb_df_graph1_2['platform'] = 'Facebook'
fb_df_graph3['platform'] = 'Facebook'

yt_df_graph1_2['platform'] = 'Youtube'
yt_df_graph3['platform'] = 'Youtube'

tw_df_graph1_2['platform'] = 'Twitter'
tw_df_graph3['platform'] = 'Twitter'

ig_df_graph1_2['platform'] = 'Instagram'
ig_df_graph3['platform'] = 'Instagram'

In [370]:
fb_list = fb_list.dropna().drop_duplicates()  
fb_df_account_info = fb_df[['account_id', 'account_display_name', 'account_username']].drop_duplicates()
fb_df_account_info.dropna(subset=['account_id'], inplace=True) 

fb_all_names = pd.merge(left = fb_df_account_info,  #fb csv
                            right = fb_list, #acct csv
                            how = 'left',
                            left_on = 'account_id',
                            right_on = 'facebook_id')

#Change empty string to NaN (to not using empty string as account name)
fb_all_names = fb_all_names.replace(r'^\s*$', np.NaN, regex=True)

fb_all_names['account_name'].fillna(fb_all_names['account_display_name'],inplace=True) 
fb_all_names['account_name'].fillna(fb_all_names['account_username'],inplace=True)  
fb_all_names = fb_all_names.groupby('account_id')['account_name'].min().reset_index() 

# display(fb_all_names[fb_all_names.groupby('account_id')['account_name'].transform('nunique') > 1])

df = pd.merge(left = fb_df_graph1_2,
                right = fb_all_names,
                how = 'left',
                on = 'account_id') 


fb_df_graph1_2_names = df

In [371]:
#Twitter
tw_list = tw_list.dropna().drop_duplicates()  
tw_df_account_info = tw_df[['account_id', 'account_display_name', 'account_username']].drop_duplicates()
tw_df_account_info.dropna(subset=['account_id'], inplace=True) 

tw_all_names = pd.merge(left = tw_df_account_info,  #tw csv
                            right = tw_list, #acct csv
                            how = 'left',
                            left_on = 'account_id',
                            right_on = 'twitter_id')


#Change empty string to NaN (to not using empty string as account name)
tw_all_names = tw_all_names.replace(r'^\s*$', np.NaN, regex=True)

tw_all_names['account_name'].fillna(tw_all_names['account_display_name'],inplace=True) 
tw_all_names['account_name'].fillna(tw_all_names['account_username'],inplace=True)  
tw_all_names = tw_all_names.groupby('account_id')['account_name'].min().reset_index() 

df = pd.merge(left = tw_df_graph1_2,
                right = tw_all_names,
                how = 'left',
                on = 'account_id') 


tw_df_graph1_2_names = df

In [372]:
yt_list = yt_list.dropna().drop_duplicates()  
yt_df_account_info = YT_df[['account_id', 'account_display_name', 'account_username']].drop_duplicates()
yt_df_account_info.dropna(subset=['account_id'], inplace=True) 

yt_all_names = pd.merge(left = yt_df_account_info,  #yt csv
                            right = yt_list, #acct csv
                            how = 'left',
                            left_on = 'account_id',
                            right_on = 'youtube_id')

#Change empty string to NaN (to not using empty string as account name)
yt_all_names = yt_all_names.replace(r'^\s*$', np.NaN, regex=True)

yt_all_names['account_name'].fillna(yt_all_names['account_display_name'],inplace=True) 
yt_all_names['account_name'].fillna(yt_all_names['account_username'],inplace=True)  
yt_all_names = yt_all_names.groupby('account_id')['account_name'].min().reset_index() 

# display(yt_all_names[yt_all_names.groupby('account_id')['account_name'].transform('nunique') > 1])

df = pd.merge(left = yt_df_graph1_2,
                right = yt_all_names,
                how = 'left',
                on = 'account_id') 


yt_df_graph1_2_names = df

In [373]:
ig_list = ig_list.dropna().drop_duplicates()  
ig_df_account_info = ig_df[['account_id', 'account_display_name', 'account_username']].drop_duplicates()
ig_df_account_info.dropna(subset=['account_id'], inplace=True) 

ig_all_names = pd.merge(left = ig_df_account_info,  #ig csv
                            right = ig_list, #acct csv
                            how = 'left',
                            left_on = 'account_id',
                            right_on = 'instagram_id')


#Change empty string to NaN (to not using empty string as account name)
ig_all_names = ig_all_names.replace(r'^\s*$', np.NaN, regex=True)

ig_all_names['account_name'].fillna(ig_all_names['account_display_name'],inplace=True) 
ig_all_names['account_name'].fillna(ig_all_names['account_username'],inplace=True)  
ig_all_names = ig_all_names.groupby('account_id')['account_name'].min().reset_index() 

# display(ig_all_names[ig_all_names.groupby('account_id')['account_name'].transform('nunique') > 1])

df = pd.merge(left = ig_df_graph1_2,
                right = ig_all_names,
                how = 'left',
                on = 'account_id') 


ig_df_graph1_2_names = df

In [374]:
#count number of platforms from account mapping

accounts_df1 = accounts_df
accounts_df1 = accounts_df1.fillna(0)
accounts_df1['no_platform_frommap'] = accounts_df1[['facebook_id','twitter_id','instagram_id','youtube_id']].astype(bool).sum(axis=1)
accounts_df1 = accounts_df1[['account_name','no_platform_frommap']]
accounts_df1 = accounts_df1.dropna().drop_duplicates() 
accounts_df1 =accounts_df1[accounts_df1['no_platform_frommap'] !=0]

In [375]:
#Union DF for graph 1&2
frames = [fb_df_graph1_2_names, yt_df_graph1_2_names, tw_df_graph1_2_names, ig_df_graph1_2_names]
overall_df_graph1_2 = pd.concat(frames) 

#Add no. of platform & influencer tier as account id 
overall_df_graph1_2['no_of_platform_count'] = overall_df_graph1_2.groupby(['account_name'])['platform'].transform('nunique')  
# overall_df_graph1_2
#if this account name is in account mapping we use no. of platform from account mapping but not we use no. of platform that counted from rawdata 
overall_df_graph1_2 = pd.merge(overall_df_graph1_2, accounts_df1, how="left", on=["account_name"])
overall_df_graph1_2.no_platform_frommap = overall_df_graph1_2.no_platform_frommap.fillna(overall_df_graph1_2.no_of_platform_count)
# and if  no. of platform in  account mapping > count from rawdata we use no. of platform from account mapping 
overall_df_graph1_2['no_of_platforms_max'] = np.where(overall_df_graph1_2['no_platform_frommap'] >= overall_df_graph1_2['no_of_platform_count'] , overall_df_graph1_2['no_platform_frommap'], overall_df_graph1_2['no_of_platform_count'])
overall_df_graph1_2 = overall_df_graph1_2.drop(overall_df_graph1_2[['no_platform_frommap', 'no_of_platform_count','no_of_platforms_max']], axis=1)
overall_df_graph1_2['no_of_platforms'] =  overall_df_graph1_2.groupby(['account_name'])['platform'].transform('nunique') 


col         = 'fan'
conditions  = [ overall_df_graph1_2[col] >= 1000000, (overall_df_graph1_2[col] < 1000000) & (overall_df_graph1_2[col] >= 500000), (overall_df_graph1_2[col] < 500000) & (overall_df_graph1_2[col] >= 50000), (overall_df_graph1_2[col] < 50000) & (overall_df_graph1_2[col] >= 10000), (overall_df_graph1_2[col] < 10000) & (overall_df_graph1_2[col] >= 1000)]
choices     = [ 'Mega', 'Macro', 'Mid-tier', 'Micro', 'Nano' ]
overall_df_graph1_2['influencer_tier'] = np.select(conditions, choices, default = 'Beginner')  

display(overall_df_graph1_2.sort_values(by = ['no_of_platforms'],ascending = False))


,account_id,fan,month,reactions,reactions_per_fan,platform,account_name,no_of_platforms,influencer_tier
29973,UCAfuDr-iaAbOSQW15Hj25qQ,25400.0,11,29564,1.163937,Youtube,THE101.WORLD,4,Micro
5233,763239420383852,355176.0,3,141374,0.398039,Facebook,MONEY BUFFALO,4,Mid-tier
14615,277414225954537,471531.0,9,81052,0.171891,Facebook,คุณแม่เลอค่า,4,Mid-tier
14616,277542568976911,63085.0,7,533,0.008449,Facebook,IREVIEW.IN.TH,4,Mid-tier
14617,277542568976911,63093.0,8,1139,0.018053,Facebook,IREVIEW.IN.TH,4,Mid-tier
...,...,...,...,...,...,...,...,...,...
14180,222118901920336,1749.0,8,3,0.001715,Facebook,TRADER GROUP เค้าเอง ก็เค้าจะตั้งชื่อแบบนี้อ่ะ,1,Nano
14184,222614711177608,548701.0,7,218521,0.398252,Facebook,หมูหยองRACING DRAG,1,Macro
14185,222614711177608,551125.0,8,346874,0.629393,Facebook,หมูหยองRACING DRAG,1,Macro
14186,222614711177608,552531.0,9,176676,0.319758,Facebook,หมูหยองRACING DRAG,1,Macro


In [376]:
#Union DF for graph 3
frames = [fb_df_graph3, yt_df_graph3, tw_df_graph3, ig_df_graph3]
overall_df_graph3_tmp = pd.concat(frames)

#Create Calendar day of week, 24 hours as keys of graph 3 (to backfill data in case of there is no transaction)
day_of_week_list = ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday', 'Sunday')
day_name_df = pd.DataFrame(day_of_week_list, columns = ['day_of_week']) 
day_name_df['key'] = 1 

hour_list = []
for i in range(24): 
    hour_list.append(i)  
hour_df = pd.DataFrame (hour_list, columns = ['hour'])
hour_df['key'] = 1  

month_list = []
for i in range(12): 
    month_list.append(i+1)  
month_df = pd.DataFrame (month_list, columns = ['month'])
month_df['key'] = 1 

platform_list = ['Facebook', 'Youtube', 'Twitter', 'Instagram']
platform_df = pd.DataFrame (platform_list, columns = ['platform'])
platform_df['key'] = 1  

all_keys = day_name_df.merge(hour_df, on = 'key') \
                            .merge(month_df, on = 'key') \
                            .merge(platform_df, on = 'key') 

all_keys

##Start with keys left join transactions (to handle null)
overall_df_graph3 = pd.merge(left = all_keys,   #keys
                            right = overall_df_graph3_tmp,  #transactions
                            how = 'left',
                             on = ['hour', 'platform', 'day_of_week', 'month'])
overall_df_graph3 = overall_df_graph3.drop(columns='key')    
overall_df_graph3 = overall_df_graph3.fillna(0)

## Main by year 

In [377]:
a = overall_df_graph1_2.groupby(['account_id', 'platform'])['month'].max() 
latest_fan = overall_df_graph1_2.merge( a, on = ['account_id', 'platform', 'month'], how = 'inner' ) 
b = latest_fan[['account_id', 'account_name', 'platform', 'fan']].drop_duplicates()  

cleaned_yr_df = overall_df_graph1_2.groupby(['account_id', 'platform'])['reactions'].sum()
cleaned_yr_df = cleaned_yr_df.to_frame().reset_index()
cleaned_yr_df = b.merge(cleaned_yr_df, on = ['account_id', 'platform']) 

#filter_criteria change \ to & becuz \ still have fan = 0  so if fan = 0 หาร react result = inf
filter_criteria = ((cleaned_yr_df['fan'] > 0) & (cleaned_yr_df['reactions'] > 0)) 
cleaned_yr_df = cleaned_yr_df.loc[filter_criteria]
cleaned_yr_df['reactions_per_fan'] = cleaned_yr_df['reactions']/cleaned_yr_df['fan']

overall_df_graph2_by_year = cleaned_yr_df 


#Add no. of platform & influencer tier as account id level
overall_df_graph2_by_year['no_of_platforms'] = overall_df_graph2_by_year.groupby(['account_name'])['platform'].transform('nunique')  
col         = 'fan'
conditions  = [ overall_df_graph2_by_year[col] >= 1000000, (overall_df_graph2_by_year[col] < 1000000) & (overall_df_graph2_by_year[col] >= 500000), (overall_df_graph2_by_year[col] < 500000) & (overall_df_graph2_by_year[col] >= 50000), (overall_df_graph2_by_year[col] < 50000) & (overall_df_graph2_by_year[col] >= 10000), (overall_df_graph2_by_year[col] < 10000) & (overall_df_graph2_by_year[col] >= 1000)]
choices     = [ 'Mega', 'Macro', 'Mid-tier', 'Micro', 'Nano' ]
overall_df_graph2_by_year['influencer_tier'] = np.select(conditions, choices, default = 'Beginner')  


In [378]:
a = overall_df_graph1_2.groupby(['account_id', 'platform'])['month'].max() 
latest_fan = overall_df_graph1_2.merge( a, on = ['account_id', 'platform', 'month'], how = 'inner' ) 
b = latest_fan[['account_id', 'account_name', 'platform', 'fan']].drop_duplicates()  

cleaned_yr_df = overall_df_graph1_2.groupby(['account_id', 'platform'])['reactions'].sum()
cleaned_yr_df = cleaned_yr_df.to_frame().reset_index()
cleaned_yr_df = b.merge(cleaned_yr_df, on = ['account_id', 'platform']) 


#filter_criteria change \ to & becuz \ still have fan = 0  so if fan = 0 หาร react result = inf
filter_criteria = ((cleaned_yr_df['fan'] > 0) & (cleaned_yr_df['reactions'] > 0)) 
cleaned_yr_df = cleaned_yr_df.loc[filter_criteria]
cleaned_yr_df['reactions_per_fan'] = cleaned_yr_df['reactions']/cleaned_yr_df['fan'] 


overall_df_graph2_by_year = cleaned_yr_df 


overall_df_graph2_by_year['no_of_platforms'] = overall_df_graph2_by_year.groupby(['account_name'])['platform'].transform('nunique')  
col         = 'fan'
conditions  = [ overall_df_graph2_by_year[col] >= 1000000, (overall_df_graph2_by_year[col] < 1000000) & (overall_df_graph2_by_year[col] >= 500000), (overall_df_graph2_by_year[col] < 500000) & (overall_df_graph2_by_year[col] >= 50000), (overall_df_graph2_by_year[col] < 50000) & (overall_df_graph2_by_year[col] >= 10000), (overall_df_graph2_by_year[col] < 10000) & (overall_df_graph2_by_year[col] >= 1000)]
choices     = [ 'Mega', 'Macro', 'Mid-tier', 'Micro', 'Nano' ]
overall_df_graph2_by_year['influencer_tier'] = np.select(conditions, choices, default = 'Beginner')  


## Breakdown 

In [379]:
#DF names of each platform
fb_df_breakdown_graph1_2['platform'] = 'Facebook'  
yt_df_breakdown_graph1_2['platform'] = 'Youtube'  
tw_df_breakdown_graph1_2['platform'] = 'Twitter'  
ig_df_breakdown_graph1_2['platform'] = 'Instagram' 

In [380]:
#fb 
fb_list = fb_list.dropna().drop_duplicates()  
fb_df_account_info = fb_df[['account_id', 'account_display_name', 'account_username']].drop_duplicates()
fb_df_account_info.dropna(subset=['account_id'], inplace=True) 

fb_all_names = pd.merge(left = fb_df_account_info,  #fb csv
                            right = fb_list, #acct csv
                            how = 'left',
                            left_on = 'account_id',
                            right_on = 'facebook_id')


#Change empty string to NaN (to not using empty string as account name)
fb_all_names = fb_all_names.replace(r'^\s*$', np.NaN, regex=True)

fb_all_names['account_name'].fillna(fb_all_names['account_display_name'],inplace=True) 
fb_all_names['account_name'].fillna(fb_all_names['account_username'],inplace=True)  
fb_all_names = fb_all_names.groupby('account_id')['account_name'].min().reset_index() 


df = pd.merge(left = fb_df_breakdown_graph1_2,
                right = fb_all_names,
                how = 'left',
                on = 'account_id')  

df = df.drop_duplicates() 

fb_df_breakdown_graph1_2_names = df

In [381]:
#Twitter
#tw 
tw_list = tw_list.dropna().drop_duplicates()  
tw_df_account_info = tw_df[['account_id', 'account_display_name', 'account_username']].drop_duplicates()
tw_df_account_info.dropna(subset=['account_id'], inplace=True) 

tw_all_names = pd.merge(left = tw_df_account_info,  #tw csv
                            right = tw_list, #acct csv
                            how = 'left',
                            left_on = 'account_id',
                            right_on = 'twitter_id')

#Change empty string to NaN (to not using empty string as account name)
tw_all_names = tw_all_names.replace(r'^\s*$', np.NaN, regex=True)

tw_all_names['account_name'].fillna(tw_all_names['account_display_name'],inplace=True) 
tw_all_names['account_name'].fillna(tw_all_names['account_username'],inplace=True)  
tw_all_names = tw_all_names.groupby('account_id')['account_name'].min().reset_index() 


df = pd.merge(left = tw_df_breakdown_graph1_2,
                right = tw_all_names,
                how = 'left',
                on = 'account_id')  

df = df.drop_duplicates() 

tw_df_breakdown_graph1_2_names = df

In [382]:
#Youtube 
yt_list = yt_list.dropna().drop_duplicates()  
yt_df_account_info = YT_df[['account_id', 'account_display_name', 'account_username']].drop_duplicates()
yt_df_account_info.dropna(subset=['account_id'], inplace=True) 

yt_all_names = pd.merge(left = yt_df_account_info,  #yt csv
                            right = yt_list, #acct csv
                            how = 'left',
                            left_on = 'account_id',
                            right_on = 'youtube_id')

#Change empty string to NaN (to not using empty string as account name)
yt_all_names = yt_all_names.replace(r'^\s*$', np.NaN, regex=True)

yt_all_names['account_name'].fillna(yt_all_names['account_display_name'],inplace=True) 
yt_all_names['account_name'].fillna(yt_all_names['account_username'],inplace=True)  
yt_all_names = yt_all_names.groupby('account_id')['account_name'].min().reset_index() 


df = pd.merge(left = yt_df_breakdown_graph1_2,
                right = yt_all_names,
                how = 'left',
                on = 'account_id')  

df = df.drop_duplicates() 

yt_df_breakdown_graph1_2_names = df

In [383]:
#IG 
#ig 
ig_list = ig_list.dropna().drop_duplicates()  
ig_df_account_info = ig_df[['account_id', 'account_display_name', 'account_username']].drop_duplicates()
ig_df_account_info.dropna(subset=['account_id'], inplace=True) 

ig_all_names = pd.merge(left = ig_df_account_info,  #ig csv
                            right = ig_list, #acct csv
                            how = 'left',
                            left_on = 'account_id',
                            right_on = 'instagram_id')

#Change empty string to NaN (to not using empty string as account name)
ig_all_names = ig_all_names.replace(r'^\s*$', np.NaN, regex=True)

ig_all_names['account_name'].fillna(ig_all_names['account_display_name'],inplace=True) 
ig_all_names['account_name'].fillna(ig_all_names['account_username'],inplace=True)  
ig_all_names = ig_all_names.groupby('account_id')['account_name'].min().reset_index() 


df = pd.merge(left = ig_df_breakdown_graph1_2,
                right = ig_all_names,
                how = 'left',
                on = 'account_id')  

df = df.drop_duplicates() 

ig_df_breakdown_graph1_2_names = df

In [384]:
#Union DF breakdown for graph 1&2
breakdown_frames = [fb_df_breakdown_graph1_2_names, yt_df_breakdown_graph1_2_names, tw_df_breakdown_graph1_2_names, ig_df_breakdown_graph1_2_names]
overall_df_breakdown_graph1_2 = pd.concat(breakdown_frames)

In [385]:
#Backfill breakdown by date graph 

df = overall_df_breakdown_graph1_2 
default_value_df = df.groupby(['month', 'platform']).sum('fan').reset_index()[['month','platform','fan']]  #.drop(['reactions', 'reactions_per_fan'], axis=1, inplace=True)
default_value_df['fan'] = (default_value_df['fan']/30).round(0)

date_list = df['created_date'].unique() 
date_df = pd.DataFrame(date_list, columns = ['created_date']) 
date_df['key'] = 1 

platform_list = ['Facebook', 'Youtube', 'Twitter', 'Instagram']
platform_df = pd.DataFrame(platform_list, columns = ['platform'])
platform_df['key'] = 1  

all_keys = platform_df.merge(date_df, on = 'key')   

all_keys['month'] = all_keys['created_date'].astype('datetime64[s]').dt.month 
all_keys

default_df = pd.merge(left = all_keys,   #keys
                    right = default_value_df,  #transactions
                    how = 'inner',
                    on = ['platform', 'month'])  
default_df = default_df.rename(columns={'fan': 'avg_fan_of_the_month'}) 
default_df.drop_duplicates()



default_df['month'] = default_df['created_date'].astype('datetime64[s]').dt.month 
default_df

overall_df_breakdown_graph1_2_tmp =   pd.merge(left = default_df,   #keys
                                        right = overall_df_breakdown_graph1_2,  #transactions
                                        how = 'left',
                                        on = ['month', 'created_date', 'platform'])  

overall_df_breakdown_graph1_2_tmp  

df = overall_df_breakdown_graph1_2_tmp 
# mask = (df['month'] == 6) & (df['created_date'] == '2020-06-30') & (df['platform'] == 'Facebook')
df['account_id'] = df['account_id'].fillna(-1) 
df['fan'] = df['fan'].fillna(df['avg_fan_of_the_month'])  
df['reactions'] = df['reactions'].fillna(0)   
df['reactions_per_fan'] = df['reactions_per_fan'].fillna(0)  
df['account_name'] = df['account_name'].fillna('Default value')  
df.drop(columns=['key','avg_fan_of_the_month'])

overall_df_breakdown_graph1_2 = df


## Export all DF to csv

In [386]:
#Main
overall_df_graph1_2.to_csv('overall_df_graph1_2.csv') 
overall_df_graph3.to_csv('overall_df_graph3.csv') 

#Main by year
overall_df_graph2_by_year.to_csv('overall_df_graph2_by_year.csv')

#Breakdown
overall_df_breakdown_graph1_2.to_csv('overall_df_breakdown_graph1_2.csv') 


In [387]:
df = pd.read_csv('overall_df_breakdown_graph1_2.csv')
df = df.drop(columns =['Unnamed: 0']) 
df 

mask = (df['month'] == 6) & (df['created_date'] == '2020-06-30') #& (df['account_id'] == -1)  
df[mask].groupby('platform').sum('fan').to_csv('check_fast.csv') #.sort_values(by='day_of_week')  

/Users/scharoensawa/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [388]:

df = overall_df_breakdown_graph1_2
mask = (df['month'] == 6) & (df['platform'] == 'Facebook') #& (df['account_id'] != -1)  #& (df['created_date'] == '2020-06-30') 
df[mask]

,platform,key,created_date,month,avg_fan_of_the_month,account_id,fan,reactions,reactions_per_fan,account_name
131085,Facebook,1,2020-06-01,6,488391528.0,100106458256720,68350.0,4682.0,0.068500,ครูพี่หมี ติวภาษาอังกฤษ
131086,Facebook,1,2020-06-01,6,488391528.0,1002461093150254,189514.0,7586.0,0.040029,ANFIELD_HUSKY
131087,Facebook,1,2020-06-01,6,488391528.0,100315104676476,44622.0,438.0,0.009816,HISTOFUN DELUXE
131088,Facebook,1,2020-06-01,6,488391528.0,1003818116369693,560065.0,2792.0,0.004985,THE SNACK
131089,Facebook,1,2020-06-01,6,488391528.0,100645088312405,88287.0,810.0,0.009175,คุยเรื่องสมอง BY หมอนุ่ม
...,...,...,...,...,...,...,...,...,...,...
162981,Facebook,1,2020-06-08,6,488391528.0,983675831677784,9343.0,225.0,0.024082,ATTACLEX
162982,Facebook,1,2020-06-08,6,488391528.0,985425661571900,209885.0,72.0,0.000343,GO JOURNEY TOGETHER
162983,Facebook,1,2020-06-08,6,488391528.0,990173817666824,166066.0,12869.0,0.077493,CIGARETTES
162984,Facebook,1,2020-06-08,6,488391528.0,997722787054306,112488.0,3526.0,0.031346,SOPITCHITCHAT โสภิตชิทแชท


In [389]:
df = overall_df_breakdown_graph1_2 
default_value_df = df.groupby(['month', 'platform']).median('fan').reset_index()[['month','platform','fan']]  #.drop(['reactions', 'reactions_per_fan'], axis=1, inplace=True)
default_value_df 

,month,platform,fan
0,1,Facebook,159505.5
1,1,Instagram,64334.0
2,1,Twitter,28385.0
3,1,Youtube,377000.0
4,2,Facebook,160561.0
5,2,Instagram,67194.0
6,2,Twitter,28510.0
7,2,Youtube,382000.0
8,3,Facebook,159403.0
9,3,Instagram,70212.0


In [390]:
date_list = df['created_date'].unique() 
date_df = pd.DataFrame(date_list, columns = ['created_date']) 
date_df['key'] = 1 

platform_list = ['Facebook', 'Youtube', 'Twitter', 'Instagram']
platform_df = pd.DataFrame(platform_list, columns = ['platform'])
platform_df['key'] = 1  

all_keys = platform_df.merge(date_df, on = 'key')   

all_keys['month'] = all_keys['created_date'].astype('datetime64[s]').dt.month 
all_keys

,platform,key,created_date,month
0,Facebook,1,2020-02-17,2
1,Facebook,1,2020-02-18,2
2,Facebook,1,2020-02-07,2
3,Facebook,1,2020-02-08,2
4,Facebook,1,2020-02-09,2
...,...,...,...,...
1459,Instagram,1,2020-12-29,12
1460,Instagram,1,2020-12-30,12
1461,Instagram,1,2020-12-07,12
1462,Instagram,1,2020-12-06,12


In [391]:
default_df = pd.merge(left = all_keys,   #keys
                    right = default_value_df,  #transactions
                    how = 'inner',
                    on = ['platform', 'month'])  
default_df = default_df.rename(columns={'fan': 'avg_fan_of_the_month'}) 
default_df.drop_duplicates()



default_df['month'] = default_df['created_date'].astype('datetime64[s]').dt.month 
default_df

,platform,key,created_date,month,avg_fan_of_the_month
0,Facebook,1,2020-02-17,2,160561.0
1,Facebook,1,2020-02-18,2,160561.0
2,Facebook,1,2020-02-07,2,160561.0
3,Facebook,1,2020-02-08,2,160561.0
4,Facebook,1,2020-02-09,2,160561.0
...,...,...,...,...,...
1459,Instagram,1,2020-12-29,12,80868.0
1460,Instagram,1,2020-12-30,12,80868.0
1461,Instagram,1,2020-12-07,12,80868.0
1462,Instagram,1,2020-12-06,12,80868.0


In [392]:
overall_df_breakdown_graph1_2 

,platform,key,created_date,month,avg_fan_of_the_month,account_id,fan,reactions,reactions_per_fan,account_name
0,Facebook,1,2020-02-17,2,471939538.0,100106458256720,68350.0,363.0,0.005311,ครูพี่หมี ติวภาษาอังกฤษ
1,Facebook,1,2020-02-17,2,471939538.0,1002461093150254,189514.0,3540.0,0.018679,ANFIELD_HUSKY
2,Facebook,1,2020-02-17,2,471939538.0,100353938099291,4371.0,125.0,0.028598,PIRATE TRADER
3,Facebook,1,2020-02-17,2,471939538.0,1003818116369693,534976.0,8817.0,0.016481,THE SNACK
4,Facebook,1,2020-02-17,2,471939538.0,101144184588390,13636.0,139.0,0.010194,นักลงทุนหมายเลข6
...,...,...,...,...,...,...,...,...,...,...
694494,Instagram,1,2020-12-31,12,95621975.0,9586853,2155.0,41.0,0.019026,UNFULL ICE
694495,Instagram,1,2020-12-31,12,95621975.0,9686816095,45414.0,14431.0,0.317765,HA.HUTATEK
694496,Instagram,1,2020-12-31,12,95621975.0,976658256,294671.0,364.0,0.001235,YINGWORLDFOOD
694497,Instagram,1,2020-12-31,12,95621975.0,988748112,2426583.0,81072.0,0.033410,SPRITEDER SPD


In [393]:
overall_df_breakdown_graph1_2_tmp =   pd.merge(left = default_df,   #keys
                                        right = overall_df_breakdown_graph1_2,  #transactions
                                        how = 'left',
                                        on = ['month', 'created_date', 'platform'])  

overall_df_breakdown_graph1_2_tmp 

,platform,key_x,created_date,month,avg_fan_of_the_month_x,key_y,avg_fan_of_the_month_y,account_id,fan,reactions,reactions_per_fan,account_name
0,Facebook,1,2020-02-17,2,160561.0,1,471939538.0,100106458256720,68350.0,363.0,0.005311,ครูพี่หมี ติวภาษาอังกฤษ
1,Facebook,1,2020-02-17,2,160561.0,1,471939538.0,1002461093150254,189514.0,3540.0,0.018679,ANFIELD_HUSKY
2,Facebook,1,2020-02-17,2,160561.0,1,471939538.0,100353938099291,4371.0,125.0,0.028598,PIRATE TRADER
3,Facebook,1,2020-02-17,2,160561.0,1,471939538.0,1003818116369693,534976.0,8817.0,0.016481,THE SNACK
4,Facebook,1,2020-02-17,2,160561.0,1,471939538.0,101144184588390,13636.0,139.0,0.010194,นักลงทุนหมายเลข6
...,...,...,...,...,...,...,...,...,...,...,...,...
694494,Instagram,1,2020-12-31,12,80868.0,1,95621975.0,9586853,2155.0,41.0,0.019026,UNFULL ICE
694495,Instagram,1,2020-12-31,12,80868.0,1,95621975.0,9686816095,45414.0,14431.0,0.317765,HA.HUTATEK
694496,Instagram,1,2020-12-31,12,80868.0,1,95621975.0,976658256,294671.0,364.0,0.001235,YINGWORLDFOOD
694497,Instagram,1,2020-12-31,12,80868.0,1,95621975.0,988748112,2426583.0,81072.0,0.033410,SPRITEDER SPD


In [394]:
df = overall_df_breakdown_graph1_2_tmp 
# mask = (df['month'] == 6) & (df['created_date'] == '2020-06-30') & (df['platform'] == 'Facebook')
df['account_id'] = df['account_id'].fillna(-1) 
df['fan'] = df['fan'].fillna(df['avg_fan_of_the_month'])  
df['reactions'] = df['reactions'].fillna(0)   
df['reactions_per_fan'] = df['reactions_per_fan'].fillna(0)  
df['account_name'] = df['account_name'].fillna('Default value')  
df.drop(columns=['key','avg_fan_of_the_month'])

overall_df_breakdown_graph1_2 = df

KeyError: 'avg_fan_of_the_month'